In [197]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import json
import pandas as pd
import traceback
# from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
# from langchain.callbacks import get_openai_callback
import PyPDF2
from langchain_google_genai import GoogleGenerativeAI


ModuleNotFoundError: No module named 'langchain_google_genai'

In [190]:
from dotenv import load_dotenv
load_dotenv() #take environment variable from .env

True

In [191]:
# Load HuggingFace API key from environment
Key = os.getenv('GOOGLE_API_KEY"')

In [192]:
print(Key)

None


In [196]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)


NameError: name 'GoogleGenerativeAI' is not defined

In [193]:
# Define response JSON format
RESPONSE_JSON = {
    '1': {'mcq': 'multiple choice question', 'options': {'a': 'option', 'b': 'option', 'c': 'option', 'd': 'option'}, 'correct': 'correct answer'},
    '2': {'mcq': 'multiple choice question', 'options': {'a': 'option', 'b': 'option', 'c': 'option', 'd': 'option'}, 'correct': 'correct answer'}
}

In [194]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [195]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

In [154]:
# Define the model and task
llm = HuggingFaceHub(
    repo_id='google/flan-t5-base',
    huggingfacehub_api_token=Key,
    model_kwargs={"temperature": 0.3},
    task="text2text-generation"  # Ensure the task is appropriate for the model
)

In [155]:
# Initialize the quiz generation chain
quiz_chain = LLMChain(
    llm=llm,
    verbose=True,
    output_key='quiz',
    prompt=quiz_generation_prompt
)

In [176]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [157]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject', 'quiz'],
    template=TEMPLATE2
)

In [158]:
# Initialize the quiz review chain
review_chain = LLMChain(
    llm=llm,
    verbose=True,
    output_key='review',
    prompt=quiz_evaluation_prompt
)

In [159]:
# Combine the quiz generation and review chains
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    output_variables=['quiz', 'review'],
    verbose=True
)

In [160]:
file_path=r'D:\mcqgen\data.txt'

In [161]:
file_path

'D:\\mcqgen\\data.txt'

In [162]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [163]:
print(TEXT)

Here are just a few examples of machine learning you might encounter every day:

Speech recognition: It is also known as automatic speech recognition (ASR), computer speech recognition, or speech-to-text, and it is a capability which uses natural language processing (NLP) to translate human speech into a written format. Many mobile devices incorporate speech recognition into their systems to conduct voice search—e.g. Siri—or improve accessibility for texting.

Customer service:  Online chatbots are replacing human agents along the customer journey, changing the way we think about customer engagement across websites and social media platforms. Chatbots answer frequently asked questions (FAQs) about topics such as shipping, or provide personalized advice, cross-selling products or suggesting sizes for users. Examples include virtual agents on e-commerce sites; messaging bots, using Slack and Facebook Messenger; and tasks usually done by virtual assistants and voice assistants.

Computer 

In [177]:
example_quiz = {
    '1': {
        'mcq': 'What is machine learning?',
        'options': {
            'a': 'A type of artificial intelligence',
            'b': 'A type of software',
            'c': 'A type of hardware',
            'd': 'A type of computer network'
        },
        'correct': 'a'
    },
    '2': {
        'mcq': 'Which of the following is a machine learning application?',
        'options': {
            'a': 'Speech recognition',
            'b': 'Text editing',
            'c': 'Image editing',
            'd': 'Web browsing'
        },
        'correct': 'a'
    }
}

# Manually review the generated quiz
print(json.dumps(example_quiz, indent=2))


{
  "1": {
    "mcq": "What is machine learning?",
    "options": {
      "a": "A type of artificial intelligence",
      "b": "A type of software",
      "c": "A type of hardware",
      "d": "A type of computer network"
    },
    "correct": "a"
  },
  "2": {
    "mcq": "Which of the following is a machine learning application?",
    "options": {
      "a": "Speech recognition",
      "b": "Text editing",
      "c": "Image editing",
      "d": "Web browsing"
    },
    "correct": "a"
  }
}


In [165]:
# Define variables
NUMBER = 5
SUBJECT = 'Machine Learning'
TONE = 'simple'

In [179]:
response = generate_evaluate_chain.apply([{
    'text': TEXT,
    'number': NUMBER,
    'subject': SUBJECT,
    'tone': TONE,
    'response_json': json.dumps(RESPONSE_JSON)  # Serialized JSON
}])

# Inspect the response format
print("Response:", response)




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Here are just a few examples of machine learning you might encounter every day:

Speech recognition: It is also known as automatic speech recognition (ASR), computer speech recognition, or speech-to-text, and it is a capability which uses natural language processing (NLP) to translate human speech into a written format. Many mobile devices incorporate speech recognition into their systems to conduct voice search—e.g. Siri—or improve accessibility for texting.

Customer service:  Online chatbots are replacing human agents along the customer journey, changing the way we think about customer engagement across websites and social media platforms. Chatbots answer frequently asked questions (FAQs) about topics such as shipping, or provide personalized advice, cross-selling products or suggesting sizes for users. Examples include virtual agents on e-commerce sites; messaging bots, usin

In [180]:
# Access the quiz and review from the chain response
quiz_output = response[0]['quiz']
review_output = response[0]['review']


In [181]:
# Display the results
print("Quiz Generated:")
print(quiz_output)


Quiz Generated:
What are the most common questions that are asked by machine learning?


In [182]:
print("\nQuiz Review:")
print(review_output)


Quiz Review:
The questions are a bit difficult to answer because they are based on a list of
